# Imports

In [2]:
import serial
import time
from pathlib import Path

In [3]:
import wave
import pyaudio

# Open Serial Port

In [4]:
ser = serial.Serial('COM5', 1.5e6, timeout=0, parity=serial.PARITY_NONE) # the baudrate of the uart dongle in verry unprecise. 1.5MHz is matched verry good.

In [8]:
ser

Serial<id=0x293b51a50f0, open=True>(port='COM5', baudrate=1500000, bytesize=8, parity='N', stopbits=1, timeout=0, xonxoff=False, rtscts=False, dsrdtr=False)

In [21]:
ser.write(bytes([0x03]))#b'U')

1

## Loop sending bytes

In [5]:
while(True):
    for byte in range(0,0x100,1):
        #print(byte)
        ser.write(bytes([byte])) #, 0x01, 0x7F, 0x3F, 0x0F,0xFF]))
        time.sleep(1e-2)

KeyboardInterrupt: 

In [44]:
ser.write(bytes([0x7E]))

1

In [ ]:
ser.close()

In [ ]:
while(True):
    time.sleep(1)
    ser.write(bytes([0x00, 0x77, 0x55]))

## Feeding sent data back to sender

# Open wav file

In [8]:
chunk_size = 1024  

In [9]:
wav_path = Path(r'D:\Sicherungen\Musik_Erwin\Knaat')

In [10]:
wav = wave.open(f=str(wav_path / 'Finjark_Master_003.wav'), mode='rb')

## Just play the song direcly in Python

In [13]:
p = pyaudio.PyAudio()  
#open stream  
stream = p.open(format = p.get_format_from_width(wav.getsampwidth()),  
                channels = wav.getnchannels(),  
                rate = wav.getframerate(),  
                output = True)  
#read data  
data = wav.readframes(chunk_size)  

#play stream  
#while data:  
#    stream.write(data)  
#    data = wav.readframes(chunk_size) 

KeyboardInterrupt: 

## Send Data of song via UART to PC and play it in Python

In [85]:
ser.close()
ser = serial.Serial(port='COM5', baudrate=256e3, timeout=5, parity=serial.PARITY_NONE, stopbits=serial.STOPBITS_ONE)

In [114]:
chunk_size = 20
data = None
wav = wave.open(f=str(wav_path / 'Finjark_Master_003.wav'), mode='rb')
p = pyaudio.PyAudio()  
#open stream  
stream = p.open(format = p.get_format_from_width(wav.getsampwidth()),  
                channels = wav.getnchannels(),  
                rate = wav.getframerate(),  
                output = True) 

ser.write(wav.readframes(chunk_size))
#data = ser.read_all()

#read data
#while data:
#    stream.write(data)
#    ser.write(wav.readframes(chunk_size))
#    data = ser.read_all()
    


#play stream  
#while data:  
#    stream.write(data)  
#    data = wav.readframes(chunk_size) 

40

## Send bytes of wav and receive aknowledge

In [46]:
wav_path = Path(r'D:\Sicherungen\Musik_Erwin\Knaat')

In [47]:
def get_transmission_chunk(wav, chunk_size):
    #frame = b'\x7E' + \
    frame = wav.readframes(chunk_size).replace(b'\x7D', b'\x7D\x5D') \
                                      .replace(b'\x7E', b'\x7D\x5E') \
            + b'\x7E'
    return frame

In [49]:
free_mem = 0
response_size = 2

wav = wave.open(f=str(wav_path / 'Finjark_Master_003.wav'), mode='rb')
p = pyaudio.PyAudio()  
#open stream  
stream = p.open(format = p.get_format_from_width(wav.getsampwidth()),  
                channels = wav.getnchannels(),  
                rate = wav.getframerate(),  
                output = True) 


ser.write(b'\x7E')
time.sleep(1e-3)
rec = ser.read(response_size).hex()
if rec == '':
    raise TimeoutError('Timout reached')
free_mem = int(rec, 16)
frame = get_transmission_chunk(wav=wav, chunk_size=free_mem // wav.getsampwidth())

#read data
while frame:
    if free_mem > 0:
        ser.write(frame)
        time.sleep(1e-3)
        rec = ser.read(response_size).hex()
        if rec == '':
            raise TimeoutError('Timout reached')
        free_mem = int(rec, 16)
        frame = get_transmission_chunk(wav=wav, chunk_size=free_mem // wav.getsampwidth())
    else:
        print('FPGA Buffer full!!')
        time.sleep(5e-3)
        ser.write(b'\x7E')
        time.sleep(1e-3)
        rec = ser.read(response_size).hex()
        if rec == '':
            raise TimeoutError('Timout reached')
        free_mem = int(rec, 16)

KeyboardInterrupt: 

In [40]:
for byte in range(0,0x100,1):
    ser.write(bytes([byte]))

In [82]:
wav.getnframes()

9879552

# Get decoded mp3 data

# Debug

In [95]:
a = b'\x7E\x7D'

In [99]:
int(a.hex(), 16)

32381

In [62]:
a.replace(b'\x7D', b'\x7D\x5D').replace(b'\x7E', b'\x7D\x5E').hex()

'7d5e7d5d'

In [55]:
type(wav.readframes(chunk_size))

bytes

In [83]:
ser.timeout

0

In [73]:
(b'\x7E' + wav.readframes(chunk_size).replace(b'\x7D', b'\x7D\x5D').replace(b'\x7E', b'\x7D\x5E') + b'\x7E').hex()

'7e0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [18]:
ser.close()

In [50]:
wav.